In [1]:
import sys
sys.path.append("../")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scorecardpipeline import *


logger = init_setting(seed=8888, logger=True)

In [2]:
# 加载数据集，标签转换为 0 和 1
target = "creditability"
data = germancredit()
data[target] = data[target].map({"good": 0, "bad": 1})

# 目前仅支持数值型变量
data = data.select_dtypes("number")
data.head(10)

,duration_in_month,credit_amount,installment_rate_in_percentage_of_disposable_income,present_residence_since,age_in_years,number_of_existing_credits_at_this_bank,number_of_people_being_liable_to_provide_maintenance_for,creditability
0,6,1169,4,4,67,2,1,0
1,48,5951,2,2,22,1,1,1
2,12,2096,2,3,49,1,2,0
3,42,7882,2,4,45,1,2,0
4,24,4870,3,4,53,2,2,1
5,36,9055,2,4,35,1,2,0
6,24,2835,3,4,53,1,1,0
7,36,6948,2,2,35,1,1,0
8,12,3059,2,4,61,1,1,0
9,30,5234,4,2,28,2,1,1


In [3]:
rule1 = Rule("duration_in_month < 4")
rule2 = Rule("credit_amount < 500")

In [4]:
rule1.report(data, target=target)

,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,duration_in_month < 4,命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,duration_in_month < 4,未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615


In [5]:
rule2.report(data, target=target)

,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,credit_amount < 500,命中,18,0.0180,15,0.0214,3,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,credit_amount < 500,未命中,982,0.9820,685,0.9786,297,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


In [6]:
(rule1 | rule2).report(data, target=target)

,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,(duration_in_month < 4) | (credit_amount < 500),命中,18,0.0180,15,0.0214,3,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,(duration_in_month < 4) | (credit_amount < 500),未命中,982,0.9820,685,0.9786,297,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


In [7]:
(rule1 & rule2).report(data, target=target)

,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,(duration_in_month < 4) & (credit_amount < 500),命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,(duration_in_month < 4) & (credit_amount < 500),未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615
